In [ ]:
import sqlite3
from datetime import datetime
from Occupation import Occupation
import json 
from pprint import pprint
from matplotlib import pyplot as plt
from get_page_length_subset_indices import get_subset_indices
from plot_page_statistics import plot_page_stats
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from bin_revision_history import month_bin_revisions
from count_contributions import contributions_by_user
from statistics import median, mean, stdev
%matplotlib widget

plt.rcParams['figure.dpi'] = 120

In [ ]:
def plot_page(occ): # for plotting diff, edits, A, employments
    
    print("lenient links:")
    pprint(occ.lenient_links)
    
    print("\n")
    print("\n")
    
    print("strict links:")
    pprint(occ.strict_links)
    plot_page_stats(occ)

### Getting the occupation-row from the database

In [ ]:
db_path = r"C:\Users\leond\Documents\Github\csh\data\data_bases\aggregated_edits.db"

con = sqlite3.connect(db_path)
cur = con.cursor()
cur.execute(f"Select * from occupations Where id = 15") # insert ID
occupations = cur.fetchall()
print(occupations[0][2])

### Wikipedia site

In [ ]:
occ = Occupation(*occupations[0])

wiki_site_name = "Auditor" # name of a given Wiki link (not the SOC profession itself!)
wiki_site = month_bin_revisions(occ.lenient_revisions)[wiki_site_name]

### Getting unique users per year

In [ ]:
def get_unique_year(year): # function that gets the unique site editors in a year
    contribs_year = []
    for date, number_edits in occ.lenient_binned_edits[wiki_site_name].items():
        if year == date.year:
            contribs_year.append(contributions_by_user(wiki_site[date]))
    unique_contribs = len(set([item[0] for sublist in contribs_year for item in sublist]))
    return unique_contribs

In [ ]:
# unique_contribs_per_year

In [ ]:
year = (next(iter(occ.lenient_binned_edits[wiki_site_name]))).year # first year in the dict
unique_contribs_per_year = []

while get_unique_year(year) != 0 or year < (list(occ.lenient_binned_edits[wiki_site_name])[-1]).year:
    unique_contribs_per_year.append([year, get_unique_year(year)])
    year += 1
    
unique_contribs_per_year

### Plotting the unique edits per year

In [ ]:
f = plt.figure()
f.set_figwidth(6.5)
f.set_figheight(3)

year = [item[0] for item in unique_contribs_per_year]
unique_contr = [item[1] for item in unique_contribs_per_year]

plt.plot(year, unique_contr)
plt.show()

### Plotting the job market data

In [ ]:
plot_page(occ)